# クイックスタート5

このノートのクイックスタートは、Microsoft の Azure AI Foundry を使って AI エージェント推論チャット アプリケーションの開発を行うための学習ノートです。

※この内容は 2025年2月時点での仕様等に基づくものです。仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。

## 前提条件

このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Azure AI Foundry プロジェクトに接続済みの Application Insights インスタンス（トレースに利用） 
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects azure-ai-inference azure-search-documents azure-core-tracing-opentelemetry azure-monitor-opentelemetry opentelemetry-sdk

## Azure AI Foundry で シンプルな AI Agent アプリを開発する

このノートでは、Azure AI Foundry SDK を使用して、AI エージェントを使った推論チャットアプリケーションを開発する方法について説明します。

Azure AI Foundry では、エージェントを実行するための Azure AI Agent Service と呼ばれるプラットフォームを提供します。Azure AI Agent Service は、エージェントの実行に必要となるコンピューティングおよびストレージのリソースを開発者が管理する必要がなく、高品質で拡張可能な AI エージェントを安全にビルド、デプロイ、スケーリングできるように設計されたフル マネージド サービスです。 

これまでの一般的な AI アプリケーションでは、クライアント側の関数呼び出しをサポートするために何百行ものコードを必要としていましたが、Azure AI Agent Service を使用することで、数行のコードで実行できるようになります。

AI Agent Service では、様々なツールを割り当てて推論モデルを使ってエージェントを実行することができます。

- ナレッジ ツール
    - Bing 検索を使用したグラウンディング （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=code-example)）
    - ファイル検索 （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/file-search?tabs=python&pivots=azure-blob-storage-code-examples)）
    - Azure AI Search （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=code-examples)）
- アクション ツール
    - コードインタープリター （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/code-interpreter?tabs=python&pivots=overview)）
    - 独自の関数 （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/function-calling?tabs=python&pivots=code-example)）
    - Azure Functions （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/azure-functions?pivots=code-example)）
    - OpenAPI で定義された API ツール （[コード例](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/how-to/tools/openapi-spec?tabs=python&pivots=code-example)）


> **注意**
> "Bing 検索を使用したグラウンディング" で対応している OpenAI モデルは以下のモデルです：
> gpt-3.5-turbo-0125、gpt-4-0125-preview、gpt-4-turbo-2024-04-09、gpt-4o-0513

このサンプルコードでは、コードインタープリターを使ったエージェントを開発します。


### 準備

このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o` または `gpt-4o-mini` モデルをプロジェクトへデプロイしてください。

エージェントをサポートするモデルには制限があります。[技術ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/concepts/model-region-support?tabs=python) を参照して、適切なモデルとバージョンのエージェントをデプロイします。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。


### チャットアプリのコードサンプル

#### 1. AI プロジェクトに接続します

[クイックスタート2](../02_simple_chat_with_prompt_template/chatprompt.ja.ipynb) で実行した手順と同様に、AI プロジェクトに接続し、SDK のトレースを有効にします。


In [ ]:
import os
from dotenv import load_dotenv
from opentelemetry import trace
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

tracer = trace.get_tracer(__name__)

load_dotenv('.env', override=True)
load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')

try:
    credential = DefaultAzureCredential()
    print(f"✅ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"❌ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print(f"✅ プロジェクトクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ プロジェクトクライアントの初期化が以下の理由で失敗しました: {str(e)}")

以下のコードを実行して、トレースを有効にします。

In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter
from azure.core.settings import settings

# Enable Azure SDK tracing with either of two lines:
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"
settings.tracing_implementation = "opentelemetry"

# Enable tracing
AIInferenceInstrumentor().instrument()
# enable logging message contents
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

application_insights_connection_string = project.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("❌ Application Insights がこのプロジェクトで有効になっていません.")
    print("==> Azure AI Foundry ポータルの \"トレース\"タブから、Application Inights を有効にしてください.")
else:
    configure_azure_monitor(connection_string=application_insights_connection_string)
    exporter = AzureMonitorTraceExporter(connection_string=application_insights_connection_string)
    print("✅ Application Insights によるトレースを有効にしました.")

#### 2. エージェントのツールとして割り当てるコードインタープリターを初期化する



[`CodeInterpreterTool`](https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.codeinterpretertool?view=azure-python-preview) ツールを使ってエージェントを作成します。

コード インタープリターにより、エージェントはサンドボックス実行環境で Python コードを記述して実行できます。
コード インタープリターを有効にすると、エージェントはコードを繰り返し実行して、より困難なコード、数学、データ分析の問題を解決できます。 

コードインタープリターの詳細については[技術ドキュメント](https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.codeinterpretertool?view=azure-python-preview)を参照します。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool
code_interpreter = CodeInterpreterTool()

#### 3. エージェントに対する指示を定義します


作成するエージェントに対する指示を以下の変数で定義します。

- `agent_name`: エージェントの名前
- `agent_instructions`: エージェントに対する手順を示すシステムメッセージ
- `message_conteent`: エージェントに最初に渡すユーザーメッセージ（テキストのみをサポート）

以下の例では、"日本人の身長と体重の平均値の BMI 値と日本人の人口全体の BMI 分布を調べ、全体分布の中の位置づけをグラフを生成して図示する" エージェントを作成します。


In [ ]:
agent_name = "bmi-calculator"

agent_instructions = "あなたは日本の測定基準 (体重 kg、身長 cm) を使用して BMI を計算する健康アナリストです。日本の平均的な身長と体重の測定値を調べてその測定値を使用します。日本人の人口全体の BMI 分布と日本人の平均 BMI が分布のどこに位置するかを示す画像を作成します。"

message_content = "日本の平均的な身長と体重の測定値の BMI を計算します。18 から 25 までの標準 BMI とされる範囲を図示しながら、日本人の人口全体の BMI 分布と平均 BMI がどこに位置するかを示す画像を作成します。画像には標準 BMI カテゴリ (やせ、正常、肥満) を含めます。画像に含めるキャプションの言語は英語にします。"

#### 4. エージェントを作成します


Azure AI Project API を使って [エージェントの操作](https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.operations.agentsoperations?view=azure-python-preview) を行います。

以下のコードでは、以下の操作を行います。

1. エージェントの名前、エージェントへ指示する手順、エージェントに渡すツールを指定して、エージェントの作成を行います
1. エージェントの（会話）スレッドを作成します
1. エージェントへのユーザー メッセージを作成します
1. エージェントを実行する
1. エージェントの実行結果のメッセージから取り出す

作成したエージェントとスレッドは、[AI Foundry ポータル](https://ai.azure.com/) の該当するプロジェクトの [エージェント] メニューから確認することができます。


In [ ]:

@tracer.start_as_current_span("quick_start_05: run_agent")
def run_agent(name, model_name, instructions, content):
    try:
        try:
            # Create an agent with code interpreter
            agent = project.agents.create_agent(
                model=model_name,
                name=name,
                instructions=instructions,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )
            print(f"[{name}] ✅ エージェントを作成しました: ID: {agent.id}")
        except Exception as e:
            print(f"[{name}] ❌ エージェントの作成に失敗しました: {str(e)}")
            return None

        # Create a thread
        thread = project.agents.create_thread()
        print(f"[{name}] ✅ スレッドを作成しました: ID: {thread.id}")

        # Request BMI analysis
        message = project.agents.create_message(thread_id=thread.id, role="user", content=content,)
        print(f"[{name}] ✅ メッセージを作成しました: {message}")

        # Process the request
        run = project.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id,)
        print(f"[{name}] ✅ エージェントの実行が完了しました: status = {run.status}")
        if run.status == "failed":
            print(f"[{name}] ❌ エージェントの実行が失敗しました: {run.last_error}")

        # Get and save any visualizations
        messages = project.agents.list_messages(thread_id=thread.id)
        print(f"[{name}] ✅ エージェントのメッセージ: {messages}")
        # Save the analysis visual to a file
        for image_content in messages.image_contents:
            file_name = f"bmi_analysis_{image_content.image_file.file_id}.png"
            project.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
            print(f"==> 分析結果をファイルに保存しました: {file_name}")

        # Print the analysis text
        if last_msg := messages.get_last_text_message_by_role("assistant"):
            print(f"==> 分析の内容: {last_msg.text.value}")

        return agent

    except Exception as e:
        print(f"[{name}] ❌ エージェント処理プロセスでエラーが発生しました: {str(e)}")
        return agent

agent = run_agent(agent_name, model_name_string, agent_instructions, message_content)

#### 5. エージェントを削除します


作成したエージェントを削除します。

作成したエージェントのスレッドも必要に応じて削除します。
このクイックスタートでは、実行履歴となるスレッドを残しておきます。


In [ ]:

@tracer.start_as_current_span("quick_start_05: delete_agent")
def delete_run_agent(agent):
    # Cleanup
    if agent:
        try:
            project.agents.delete_agent(agent.id)
            print(f"[{agent_name}] ✅ エージェントを削除しました.")
        except Exception as e:
            print(f"[{agent_name}] ❌ すでにエージェントは削除されています: {str(e)}")

delete_run_agent(agent)